In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
filelocation = r"E:\Rnw\python\Final_project\customer_credit_risk_500k.csv"
df = pd.read_csv(filelocation)
df.head()

,customer_id,age,gender,region,education_level,employment_type,annual_income,loan_amount,loan_purpose,credit_score,repayment_history,transaction_count,spending_ratio,join_date,default_flag
0,100000,59.0,Female,East,Secondary,Salaried,2.400795e+05,92740.780161,Education,784.170707,1,53,41.296668,2016-10-06,0
1,100001,49.0,Female,South,Graduate,Self-Employed,4.389233e+05,64315.332332,Car,589.698466,1,62,12.192453,2022-04-01,0
2,100002,35.0,Female,North,Secondary,Salaried,4.241221e+05,632481.938730,Home,625.789192,2,45,23.680727,2024-12-24,0
3,100003,63.0,Female,North,Secondary,Salaried,3.222749e+05,118465.971789,Car,627.479500,2,57,32.661094,2021-03-17,0
4,100004,28.0,Female,East,Graduate,Self-Employed,1.371926e+06,131836.274600,Car,803.177705,1,46,15.402834,2024-04-25,1


In [11]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_id        500000 non-null  int64  
 1   age                475000 non-null  float64
 2   gender             475000 non-null  str    
 3   region             500000 non-null  str    
 4   education_level    500000 non-null  str    
 5   employment_type    475000 non-null  str    
 6   annual_income      475000 non-null  float64
 7   loan_amount        500000 non-null  float64
 8   loan_purpose       500000 non-null  str    
 9   credit_score       475000 non-null  float64
 10  repayment_history  500000 non-null  int64  
 11  transaction_count  500000 non-null  int64  
 12  spending_ratio     500000 non-null  float64
 13  join_date          500000 non-null  str    
 14  default_flag       500000 non-null  int64  
dtypes: float64(5), int64(4), str(6)
memory usage: 57.2 MB


In [12]:
df.describe()

,customer_id,age,annual_income,loan_amount,credit_score,repayment_history,transaction_count,spending_ratio,default_flag
count,500000.000000,475000.000000,4.750000e+05,5.000000e+05,475000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,349999.500000,45.011236,5.497334e+05,2.242788e+05,649.870773,1.999366,50.008804,40.086244,0.151130
std,144337.711635,14.134525,4.394833e+05,2.121152e+05,79.530056,1.414278,7.063937,14.875279,0.358176
min,100000.000000,21.000000,1.570025e+04,3.331737e+03,300.000000,0.000000,20.000000,5.000000,0.000000
25%,224999.750000,33.000000,2.962467e+05,9.522698e+04,596.004894,1.000000,45.000000,29.925094,0.000000
50%,349999.500000,45.000000,4.453717e+05,1.629643e+05,649.927856,2.000000,50.000000,40.016132,0.000000
75%,474999.250000,57.000000,6.706867e+05,2.794298e+05,703.837988,3.000000,55.000000,50.155434,0.000000
max,599999.000000,69.000000,1.622835e+07,7.449222e+06,850.000000,13.000000,85.000000,100.000000,1.000000


In [ ]:
colunm_name = df.columns


0         59.0
1         49.0
2         35.0
3         63.0
4         28.0
          ... 
499995    31.0
499996    63.0
499997    63.0
499998    31.0
499999    39.0
Name: age, Length: 500000, dtype: float64